# Ray Serve - Integration with FastAPI

© 2019-2022, Anyscale. All Rights Reserved

Ray Serve integrates well with other common [web serving frameworks](https://docs.ray.io/en/latest/serve/tutorials/web-server-integration.html). 

In this tutorial, we’ll cover how to deploy [XGBoost](https://xgboost.readthedocs.io/en/stable/) with [FastAPI](https://fastapi.tiangolo.com/) and Ray Serve. We'll use a simple XGBboost classifcation model to train, deploy it on Ray Serve , and access it via HTTP request on a FastAPI endpoint. 

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.

<img src="https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png" width="40%" height="20%"> 

This XGBoost model will be trained to predict the onset of diabetes using the pima-indians-diabetes dataset from the [UCI Machine Learning Repository website](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv). This small dataset contains several numerical medical variables of eight different features related to diabetes, in addition to one target variable — Outcome. So, we’ll use XGBoost to model and solve a simple prediction problem. This tutorial is derived from our [blog](https://www.anyscale.com/blog/deploying-xgboost-models-with-ray-serve).

Let's see how easy it is!



In [1]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

import ray
from fastapi import FastAPI, Request
from ray import serve

### Load the data

In [2]:
# Load the data
dataset = np.loadtxt('data/pima-indians-diabetes.data.csv', delimiter=",")
# split data into X and y
X = dataset[:, 0:8]
y = dataset[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

### Define a utility function:
 * Creates XGBoost classifier
 * trains, fits, and saves the model
 * returns a model

In [3]:
def create_and_save_model():
    # Instantiate a model, fit and train
    xgb_model = XGBClassifier(use_label_encoder=False)
    xgb_model.fit(X_train, y_train)

    # saving the model
    with open('xgb_model.pkl', 'wb') as f:
        pickle.dump(xgb_model, f)

    return xgb_model

### Create, fit and predict XGBoost model

In [4]:
model = create_and_save_model()
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 74.02%


### Create a Ray Serve Deployment with FastAPI

In [ ]:
app = FastAPI()
ray.init(namespace="xgbregressor")
serve.start()

2022-07-05 16:14:48,723	ERROR services.py:1488 -- Failed to start the dashboard: Failed to start the dashboard, return code 1
Failed to read dashboard log: [Errno 2] No such file or directory: '/tmp/ray/session_2022-07-05_16-14-47_425380_45125/logs/dashboard.log'
2022-07-05 16:14:48,724	ERROR services.py:1489 -- Failed to start the dashboard, return code 1
Failed to read dashboard log: [Errno 2] No such file or directory: '/tmp/ray/session_2022-07-05_16-14-47_425380_45125/logs/dashboard.log'
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.9/site-packages/ray/_private/services.py", line 1451, in start_dashboard
    with open(dashboard_log, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ray/session_2022-07-05_16-14-47_425380_45125/logs/dashboard.log'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.9/site-packages/ray/_private/services.py", line 14

In [7]:
@serve.deployment(num_replicas=2, route_prefix="/regressor")
@serve.ingress(app)
class XGBModel:
    def __init__(self):
        # loading the model
        with open("xgb_model.pkl", "rb") as f:
            self.model = pickle.load(f)
        print("Pickled XGBoost model loaded")

    @app.post("/")
    async def predict(self, starlette_request: Request):
        payload = await starlette_request.json()
        print("Worker: received starlette request with data", payload)

        prediction = round(self.model.predict([np.array(list(payload.values()))])[0])
        
        return {"result": prediction}

In [8]:
XGBModel.deploy()

2022-03-16 16:55:47,937	INFO api.py:262 -- Updating deployment 'XGBModel'. component=serve deployment=XGBModel
(ServeController pid=66452) 2022-03-16 16:55:48,006	INFO deployment_state.py:920 -- Adding 2 replicas to deployment 'XGBModel'. component=serve deployment=XGBModel
(XGBModel pid=66446) /usr/local/anaconda3/envs/anyscale-academy/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(XGBModel pid=66446)   from pandas import MultiIndex, Int64Index
(XGBModel pid=66449) /usr/local/anaconda3/envs/anyscale-academy/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(XGBModel pid=66449)   from pandas import MultiIndex, Int64Index
2022-03-16 16:55:48,980	INFO api.py:274 -- Deployment 'XGBMod

(XGBModel pid=66446) Pickled XGBoost model loaded
(XGBModel pid=66449) Pickled XGBoost model loaded


### List current deployments

In [9]:
print(serve.list_deployments())

{'XGBModel': Deployment(name=XGBModel,version=None,route_prefix=/regressor)}


### Send request to the FastAPI endpoint

In [10]:
import requests

sample_request_inputs = [
    {"Pregnancies": 6,
     "Glucose": 148,
     "BloodPressure": 72,
     "SkinThickness": 35,
     "Insulin": 0,
     "BMI": 33.6,
     "DiabetesPedigree": 0.625,
     "Age": 50,
    },
    {"Pregnancies": 10,
      "Glucose": 168,
      "BloodPressure": 74,
      "SkinThickness": 0,
      "Insulin": 0,
      "BMI": 38.0,
      "DiabetesPedigree": 0.537,
      "Age": 34,
    },
    {"Pregnancies": 10,
     "Glucose": 39,
     "BloodPressure": 80,
     "SkinThickness": 0,
     "Insulin": 0,
     "BMI": 27.1,
     "DiabetesPedigree": 1.441,
     "Age": 57,
     },
     {"Pregnancies": 1,
      "Glucose": 103,
      "BloodPressure": 30,
      "SkinThickness": 38,
      "Insulin": 83,
      "BMI": 43.3,
      "DiabetesPedigree": 0.183,
      "Age": 33,
     }
    ]

In [11]:
# Iterate our requests
for sri in sample_request_inputs:
    response = requests.post("http://localhost:8000/regressor/", json=sri)
    print(response.text)

{"result":1}
{"result":1}
{"result":0}
(XGBModel pid=66446) Worker: received starlette request with data {'Pregnancies': 6, 'Glucose': 148, 'BloodPressure': 72, 'SkinThickness': 35, 'Insulin': 0, 'BMI': 33.6, 'DiabetesPedigree': 0.625, 'Age': 50}
(XGBModel pid=66446) Worker: received starlette request with data {'Pregnancies': 10, 'Glucose': 39, 'BloodPressure': 80, 'SkinThickness': 0, 'Insulin': 0, 'BMI': 27.1, 'DiabetesPedigree': 1.441, 'Age': 57}
(XGBModel pid=66449) Worker: received starlette request with data {'Pregnancies': 10, 'Glucose': 168, 'BloodPressure': 74, 'SkinThickness': 0, 'Insulin': 0, 'BMI': 38.0, 'DiabetesPedigree': 0.537, 'Age': 34}
(XGBModel pid=66449) Worker: received starlette request with data {'Pregnancies': 1, 'Glucose': 103, 'BloodPressure': 30, 'SkinThickness': 38, 'Insulin': 83, 'BMI': 43.3, 'DiabetesPedigree': 0.183, 'Age': 33}
{"result":0}


In [12]:
ray.shutdown()